In [11]:
import pandas as pd
from tqdm import tqdm
from peewee import SqliteDatabase, fn
from playhouse.shortcuts import model_to_dict

from database import Page

In [2]:
path_data = '/Volumes/ExternalSSD/FakeNewsRecognition/'
peewee_database_postgres = SqliteDatabase(path_data + 'news_cleaned_postgres.db')
peewee_database_2018_01_29_and_postgres = SqliteDatabase(path_data + 'news_cleaned_2018_01_29+postgres.db')
peewee_database_2018_02_13  = SqliteDatabase(path_data + 'news_cleaned_2018_01_29+postgres+nytimes+webhose.db')

In [3]:
Page._meta.database = peewee_database_2018_02_13
Page.select(fn.Max(Page.id)).scalar()

9878296

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Analytics" data-toc-modified-id="Analytics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analytics</a></span></li><li><span><a href="#Merge-news_cleaned_2018_01_29-with-news_cleaned_postgres" data-toc-modified-id="Merge-news_cleaned_2018_01_29-with-news_cleaned_postgres-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Merge news_cleaned_2018_01_29 with news_cleaned_postgres</a></span></li></ul></div>

# Analytics

In [7]:
def fetch_pages(last_id, batch_size, db):
    while True:
        Page._meta.database = db
        db.connect(reuse_if_open=True)
        
        pages = list(Page.select(Page.id, Page.domain)
                     .where((Page.id > last_id) & (Page.id <= (last_id + batch_size)))
                     .order_by(Page.id.asc()))

        if len(pages) <= 0:
            return

        for page in pages:
            yield page

            last_id = page.id

In [8]:
domains_counter = {}
with tqdm() as progress:
    for page in fetch_pages(0, 10 * 1000, peewee_database_2018_02_13):
        domains_counter.setdefault(page.domain, 0)
        domains_counter[page.domain] += 1
        progress.update()

9878296it [33:04, 4977.47it/s] 


In [9]:
domains_counter_list = list(domains_counter.items())
domains_counter_sorted = sorted(domains_counter_list, key=lambda x: x[1], reverse=True)
domains_counter_sorted

[('nytimes.com', 1542967),
 ('dailykos.com', 1088536),
 ('beforeitsnews.com', 783308),
 ('nationalreview.com', 484009),
 ('express.co.uk', 477681),
 ('sputniknews.com', 320057),
 ('abovetopsecret.com', 238640),
 ('wikileaks.org', 199553),
 ('pjmedia.com', 140003),
 ('www.newsmax.com', 113766),
 ('pravda.ru', 104717),
 ('www.ammoland.com', 93863),
 ('wikispooks.com', 86995),
 ('www.amazon.com', 83227),
 ('lifezette.com', 82998),
 ('twitchy.com', 81575),
 ('katehon.com', 75923),
 ('dailycaller.com', 74320),
 ('investmentwatchblog.com', 71391),
 ('naturalnews.com', 70292),
 ('breitbart.com', 65678),
 ('rawstory.com', 62463),
 ('thepostemail.com', 61029),
 ('conservapedia.com', 56595),
 ('presstv.com', 54513),
 ('lifenews.com', 53524),
 ('thedailysheeple.com', 52712),
 ('us.blastingnews.com', 50825),
 ('truthandaction.org', 49205),
 ('unz.com', 48434),
 ('zerohedge.com', 47694),
 ('liberalamerica.org', 46730),
 ('washingtonexaminer.com', 46683),
 ('thesleuthjournal.com', 46366),
 ('thinkpr

In [12]:
df_websites = pd.read_excel('data/7_opensources_co/websites_with_results.xlsx')
domains = [u for u in df_websites.url.values]

domain_type = {}
websites_url = df_websites.url.values
websites_type = df_websites.type.values
for i, url in enumerate(websites_url):
    domain_type[url] = websites_type[i]

In [16]:
count_domains_in_websites = 0
for domain, _ in domains_counter_sorted:
    if domain in websites_url:
        count_domains_in_websites += 1
        
print(count_domains_in_websites, 'out of', len(websites_url))

745 out of 1001


# Merge news_cleaned_2018_01_29 with news_cleaned_postgres

In [15]:
def insert_bulk(pages, db):
    Page._meta.database = db
    db.connect(reuse_if_open=True)
    
    with db.atomic():
        Page.insert_many(pages).execute()

In [16]:
batch_size = 50
pages_to_insert = []
with tqdm() as progress:
    for page in fetch_pages(0, batch_size, peewee_database_postgres):
        _page = model_to_dict(page)
        del _page['id']
        
        pages_to_insert.append(_page)
        progress.update()
        
        if len(pages_to_insert) > batch_size:
            insert_bulk(pages_to_insert, peewee_database_2018_01_29_and_postgres)
            pages_to_insert = []   
        
insert_bulk(pages_to_insert, peewee_database_2018_01_29_and_postgres)

6755636it [1:34:54, 1186.28it/s]
